In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn 
import csv
import json

In [7]:
data={"users":[]}
with open('DATA.json', 'w') as outfile:
    json.dump(data, outfile)

In [86]:
import json
def write_json(new_data, filename='DATA.json'):
    try:
        # Open the file in read mode
        with open(filename, 'r') as file:
            # Load existing data into a dict
            file_data = json.load(file)
    except FileNotFoundError:
        # If the file doesn't exist, create an empty dictionary
        file_data = {"users": []}

    # Append new_data to the "users" key
    file_data["users"].append(new_data)

    # Open the file in write mode
    with open(filename, 'w') as file:
        # Write the updated data back to the file
        json.dump(file_data, file, indent=4)


In [9]:
df_tr=pd.read_csv('Medical_dataset/Training.csv')

In [10]:
df_tr.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [11]:
df_tr.shape

(4920, 133)

In [13]:
df_tt=pd.read_csv('Medical_dataset/Testing.csv')

In [14]:
df_tt.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [87]:
symp = []
disease = []

for i in range(len(df_tr)):
    symptoms = df_tr.columns[df_tr.iloc[i] == 1].tolist()
    symp.append(symptoms)
    disease.append(df_tr.iloc[i, -1])


In [16]:
symp[0]

['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches']

In [17]:
disease[50]

'Peptic ulcer diseae'

# Preprocessing


In [18]:
all_symp_col=list(df_tr.columns[:-1])
def clean_symp(sym):
    return sym.replace('_',' ').replace('.1','').replace('(typhos)','').replace('yellowish','yellow').replace('yellowing','yellow') 

In [19]:
all_symp=[clean_symp(sym) for sym in (all_symp_col)]

In [89]:
import nltk
nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn

first_syns = []
second_syns = []

for sym in all_symp:
    # Checking if the symptom has synsets
    if not wn.synsets(sym):
        first_syns.append(sym)  
    else:
        second_syns.append(sym)  

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/srikarsetty/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [91]:
len(first_syns)

99

In [92]:
len(second_syns)

33

In [24]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
nlp = spacy.load('en_core_web_sm')

In [93]:
#lemmatization
def preprocess(doc):
    nlp_doc = nlp(doc)
    d = []
    for token in nlp_doc:
        # Check if the lowercase version of the token is not a stop word and consists of alphabetic characters only
        if not token.text.lower() in STOP_WORDS and token.text.isalpha():
            # Lemmatize the token to its base form and append the lowercase lemma to the list
            d.append(token.lemma_.lower())
    return ' '.join(d)

In [94]:
preprocess("skin peeling")

'skin peel'

In [96]:
#preprocessing all the symptoms
all_symp_pr = []

for sym in all_symp:
    preprocessed_sym = preprocess_sym(sym)
    all_symp_pr.append(preprocessed_sym)

In [97]:
#creating a dictionary where keys are the preprocessed symptoms and values are the corresponding original symptoms
col_dict = dict(zip(all_symp_pr, all_symp_col))

# Syntactic Similarity

In [98]:
# defining a function that calculates jaccard similarity coefficient between two sets
def jaccard_set(str1, str2):
    list1=str1.split(' ')
    list2=str2.split(' ')
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [100]:

def syntactic_similarity(symp_t, corpus):
    most_sim = []
    poss_sym = []
    
    for symp in corpus:
        d = jaccard_set(symp_t, symp)
         # Storing the symptom and its similarity coefficient
        most_sim.append((symp, d)) 
    
    # Sort by similarity coefficient in descending order
    most_sim.sort(key=lambda x: x[1], reverse=True)
    
    similar_symptoms = []
    for symp, sim in most_sim:
        if DoesExist(symp_t):
            return 1, [symp]  
        if symp != symp_t and sim != 0 and symp not in poss_sym:
            poss_sym.append(symp)  
            similar_symptoms.append((symp, sim))
    
    if len(similar_symptoms):
        return 1, similar_symptoms  
    else:
        return 0, None 


In [32]:
import itertools
#Returns all the subsets of the set
def powerset(seq):
    seq_list = list(seq)
    subsets = [[]]
    for item in seq_list:
        subsets.extend([subset + [item] for subset in subsets])
    return subsets

In [102]:
def sort(a):
    # Sorting the list based on the lengths of the strings
    sorted_list = sorted(a, key=len)
    return sorted_list


In [34]:
# find all permutations of a list
def permutations(s):
    permutations = list(itertools.permutations(s))
    return([' '.join(permutation) for permutation in permutations])

In [103]:
def DoesExist(txt):
    # Split the input text into individual words
    txt = txt.split(' ')
    # Generating all possible combinations of words using powerset function
    combinations = [x for x in powerset(txt)]
    
    # Iterate over each combination
    for comb in combinations:
        joined_comb = ' '.join(comb)
        # Check if the joined combination exists in all_symp_pr
        if joined_comb in all_symp_pr:
            # If a matching symptom string is found, return it
            return joined_comb
    
    # If no matching symptom string is found, return False
    return False

    

In [104]:
DoesExist('worried')

False

In [105]:
preprocess('really worried')

'worried'

In [106]:
syntactic_similarity(preprocess('nervous') ,all_symp_pr)

(0, None)

In [107]:
import re

def check_pattern(inp, dis_list):
    pred_list = []
    pattern = re.compile(inp)

    for item in dis_list:
        # Check if the pattern matches the item
        if pattern.search(item):
            # If a match is found, append the item to the predicted list
            pred_list.append(item)
    # If any predicted matches are found, return them
    if pred_list:
        return 1, pred_list
    else:
        # If no matches are found, return None
        return 0, None


In [113]:
check_pattern('eye',all_symp_pr)

(1,
 ['sink eye',
  'pain eye',
  'yellow eye',
  'redness eye',
  'puffy face eye',
  'water eye'])

# Semantic Similarity

In [114]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize

def WSD(word, context):
    tokens = word_tokenize(context)
    sense = lesk(tokens, word)

    return sense

In [42]:
def semanticD(doc1,doc2):
    doc1_p=preprocess(doc1).split(' ')
    doc2_p=preprocess_sym(doc2).split(' ')
    score=0
    for tock1 in doc1_p:
        for tock2 in doc2_p:
            syn1 = WSD(tock1,doc1)
            syn2 = WSD(tock2,doc2)
            #syn1=wn.synset(t)
            if syn1 is not None and syn2 is not None :
                x=syn1.wup_similarity(syn2)
                if x is not None and x>0.1:
                    score+=x
    return score/(len(doc1_p)*len(doc2_p))

In [43]:
semanticD('anxiety','nervous')

0.2

In [44]:
syna=wn.synsets('anxiety')
syna[0].definition()

'(psychiatry) a relatively permanent state of worry and nervousness occurring in a variety of mental disorders, usually accompanied by compulsive behavior or attacks of panic'

In [116]:
syn2=wn.synsets('fatigue')
syn2[0].definition()

'temporary loss of strength and energy resulting from hard physical or mental work'

In [119]:
syn2[0].wup_similarity(syna[0])

0.6153846153846154

In [121]:
fatigue_synsets = wn.synsets("fatigue") 
nervous_synsets = wn.synsets("nervous") 
path=[]
wup=[]



for s1 in fatigue_synsets:
    for s2 in nervous_synsets:
        path.append(s1.path_similarity(s2))#path similarity
        wup.append(s1.wup_similarity(s2))#wu-palmer similarity
    
        

pd.DataFrame([path,wup],["path","wup"])

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
path,0.125000,0.125000,0.125000,0.125000,0.125000,0.111111,0.111111,0.111111,0.111111,0.111111,...,0.142857,0.142857,0.142857,0.142857,0.142857,0.200000,0.200000,0.200000,0.200000,0.200000
wup,0.222222,0.222222,0.222222,0.222222,0.222222,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.250000,0.250000,0.250000,0.250000,0.250000,0.333333,0.333333,0.333333,0.333333,0.333333


In [124]:
import nltk
nltk.download('punkt')

#Finding the most semantically similar symptom
def semantic_similarity(symp_t, corpus):
    max_sim = 0
    most_sim = None
    
    # Calculating semantic similarity for each symptom in the corpus
    for symp in corpus:
        sim_score = semanticD(symp_t, symp)
        
        # Updating max_sim and most_sim if current similarity score is higher
        if sim_score > max_sim:
            max_sim = sim_score
            most_sim = symp
    
    # Returning the most similar symptom and its similarity score
    return max_sim, most_sim


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/srikarsetty/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [125]:
semantic_similarity('nervous',all_symp_pr)

(0.5, 'toxic look')

In [126]:
all_symp_pr.sort()

In [127]:
all_symp_pr

['abdominal pain',
 'abnormal menstruation',
 'acidity',
 'acute liver failure',
 'alter sensorium',
 'anxiety',
 'belly pain',
 'blackhead',
 'bladder discomfort',
 'blister',
 'blood sputum',
 'bloody stool',
 'blur distorted vision',
 'breathlessness',
 'brittle nail',
 'bruise',
 'burn micturition',
 'chest pain',
 'chill',
 'cold hand feet',
 'coma',
 'congestion',
 'constipation',
 'continuous feel urine',
 'continuous sneezing',
 'cough',
 'cramp',
 'dark urine',
 'dehydration',
 'depression',
 'diarrhoea',
 'dischromic patch',
 'distention abdoman',
 'dizziness',
 'dry tingle lip',
 'enlarge thyroid',
 'excessive hunger',
 'extra marital contact',
 'family history',
 'fast heart rate',
 'fatigue',
 'fluid overload',
 'fluid overload',
 'foul smell urine',
 'headache',
 'high fever',
 'hip joint pain',
 'history alcohol consumption',
 'increase appetite',
 'indigestion',
 'inflammatory nail',
 'internal itching',
 'irregular sugar level',
 'irritability',
 'irritation anus',
 'i

In [128]:
from itertools import chain
from nltk.corpus import wordnet

def suggest_syn(symptom):
    suggested_synonyms = set()
    synonyms = wordnet.synsets(symptom)
    
    # Collecting lemma names from synsets
    lemma_names = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
    # Calculate semantic similarity with symptoms in the corpus for each synonym
    for synonym in lemma_names:
        similarity_score, most_similar_symptom = semantic_similarity(synonym, all_symp_pr)
        # If similarity score is non-zero, add the most similar symptom as a suggested synonym
        if similarity_score != 0:
            suggested_synonyms.add(most_similar_symptom)
    
    return list(suggested_synonyms)


In [132]:
suggest_syn('pain')

['pain',
 'muscle waste',
 'toxic look',
 'bruise',
 'anxiety',
 'unsteadiness',
 'constipation',
 'malaise']

In [133]:
#One Hot Encoding
import numpy as np
import pandas as pd

def OHV(client_symptoms, all_symptoms):
    # Initialize an array with zeros
    one_hot_vector = np.zeros(len(all_symptoms))
    
    # Set values to 1 for client symptoms
    for symptom in client_symptoms:
        if symptom in all_symptoms:
            index = all_symptoms.index(symptom)
            one_hot_vector[index] = 1
    
    # Create DataFrame from the one-hot encoded vector
    one_hot_df = pd.DataFrame([one_hot_vector], columns=all_symptoms)
    
    return one_hot_df

In [55]:
def contains(small, big):
    a=True
    for i in small:
        if i not in big:
            a=False
    return a

In [134]:
def possible_diseases(l):
    posible_dis=[]
    for dis in set(disease):
        if contains(l,symVONdisease(df_tr,dis)):
            possible_dis.append(dis)
    return possible_dis

In [135]:
set(disease)

{'(vertigo) Paroymsal  Positional Vertigo',
 'AIDS',
 'Acne',
 'Alcoholic hepatitis',
 'Allergy',
 'Arthritis',
 'Bronchial Asthma',
 'Cervical spondylosis',
 'Chicken pox',
 'Chronic cholestasis',
 'Common Cold',
 'Dengue',
 'Diabetes ',
 'Dimorphic hemmorhoids(piles)',
 'Drug Reaction',
 'Fungal infection',
 'GERD',
 'Gastroenteritis',
 'Heart attack',
 'Hepatitis B',
 'Hepatitis C',
 'Hepatitis D',
 'Hepatitis E',
 'Hypertension ',
 'Hyperthyroidism',
 'Hypoglycemia',
 'Hypothyroidism',
 'Impetigo',
 'Jaundice',
 'Malaria',
 'Migraine',
 'Osteoarthristis',
 'Paralysis (brain hemorrhage)',
 'Peptic ulcer diseae',
 'Pneumonia',
 'Psoriasis',
 'Tuberculosis',
 'Typhoid',
 'Urinary tract infection',
 'Varicose veins',
 'hepatitis A'}

In [136]:
def symVONdisease(df, disease):
    # Filter the DataFrame to select rows where 'prognosis' equals the given disease
    filtered_df = df[df['prognosis'] == disease]
    
    # Get symptom columns where the value is 1
    symptom_columns = filtered_df.columns[filtered_df.eq(1).any()].tolist()
    
    return symptom_columns


In [148]:
symVONdisease(df_tr,'Psoriasis')

['skin_rash',
 'joint_pain',
 'skin_peeling',
 'silver_like_dusting',
 'small_dents_in_nails',
 'inflammatory_nails']